PART 1: BUILD DATA FRAME

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Scraping Data from wikipedia
import requests # library to handle requests
from bs4 import BeautifulSoup 

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())

In [3]:
# Get table
mytable = soup.find('table',{'class':'wikitable sortable'})

# Get headings
ths = mytable.find_all('th')
ths
headings = [th.text.strip() for th in ths]

# Get Rows into list
trs = mytable.find_all('tr')
mylist=[]

for tr in mytable.find_all('tr'):
    tds = tr.find_all('td')
    aa = [td.text.strip() for td in tds]
    mylist.append(aa)    
del mylist[0]

# Instantiate the dataframe
mydf = pd.DataFrame(columns=headings)

# Fill data frame
for l in mylist:
    pc=l[0]
    bor=l[1]
    neigh=l[2]
    
    mydf = mydf.append({'Postal Code': pc,
                            'Borough': bor,
                            'Neighborhood': neigh        
                            }, ignore_index=True)

# Clean Data
mydf.replace('Not assigned', np.nan, inplace = True)
mydf.dropna(subset = ["Borough"],inplace = True)
mydf.reset_index(inplace = True)
mydf.drop(['index'], axis=1, inplace = True)
print(mydf.head())

  Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


In [4]:
#mydf.loc[mydf['Postal Code'] == 'M9V']

In [5]:
print('Number of Rows: {}'.format(mydf.shape[0]))

Number of Rows: 103


PART 2: DATA FRAME WITH LATITUDE AND LONGITUDE COORDINATES

In [16]:
# Import csv file with Postal code Coordinates
url = 'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
postalCodeCoordinates = pd.read_csv(url,',')

# Merge both data
myDfFull = pd.merge(mydf, postalCodeCoordinates, on=['Postal Code'])

In [24]:
# myDfFull.loc[myDfFull['Postal Code'] == 'M9V']
myDfFull.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
